In [21]:
from datetime import datetime
import backtrader as bt
# 导入pyfolio 包
import pyfolio as pf


# 创建策略类
class SmaCross(bt.Strategy):
    # 定义参数
    params = dict(period=5)  # 移动平均期数

    # 日志函数
    def log(self, txt, dt=None):
        '''日志函数'''
        dt = dt or self.datas[0].datetime.datetime(0)
        t_log= ''
        t_log += ' {:s}'.format(dt.strftime('%a %Y-%m-%d %H:%M:%S'))
        t_log += ' '+ txt
        print(t_log)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 订单状态 submitted/accepted，无动作
            return
        # 订单完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('买单执行, %.2f' % order.executed.price)
            elif order.issell():
                self.log('卖单执行, %.2f' % order.executed.price)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单 Canceled/Margin/Rejected')

    # 记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self, trade):
        if trade.isclosed:
            print('毛收益 %0.2f, 扣佣后收益 % 0.2f, 佣金 %.2f' %
                  (trade.pnl, trade.pnlcomm, trade.commission))

    def __init__(self):
        self.myenterdate : datetime = None  # 入场时间
        self.dtdate = self.data.datetime.datetime(0)
        # 移动平均线指标
        self.move_average = bt.ind.MovingAverageSimple(
            self.data, period=self.params.period)
        # 交叉信号指标
        self.crossover = bt.ind.CrossOver(self.data, self.move_average)

    def next(self):
        if not self.position:  # 还没有仓位
            # 当日收盘价上穿5日均线，创建买单，买入100股
            if self.crossover > 0:
                self.myenterdate = self.dtdate
                self.log('创建买单{:}'.format(self.myenterdate.strftime('%Y-%m-%d %H:%M:%S')), dt=self.dtdate)
                self.buy(size=100)
        # 有仓位，并且当日收盘价下破5日均线，创建卖单，卖出100股
        elif self.crossover < 0:
            self.log('创建卖单')
            self.sell(size=100)


##########################
# 主程序开始
#########################

# 创建大脑引擎对象
cerebro = bt.Cerebro()


# 数据文件路径
datapath = '../Data/2006-min-005.txt'

# 创建行情数据对象，加载数据
data = bt.feeds.GenericCSVData(
    dataname=datapath,
    datetime=0,  # 日期所在列
    time=1,  # 时间所在列
    open=2,  # 开盘价所在列
    high=3,  # 最高价所在列
    low=4,  # 最低价所在列
    close=5,  # 收盘价价所在列
    volume=6,  # 成交量所在列
    openinterest=-1,  # 无未平仓量列
    dtformat='%Y-%m-%d',  # 日期格式
    tmformat='%H:%M:%S',  # 时间格式
    timeframe=bt.TimeFrame.Minutes,  # 数据周期压缩周期
    compression=5,  # 合成bar的周期数
    fromdate=datetime(2006, 1, 2),  # 起始日
    todate=datetime(2006, 1, 3))  # 结束日

cerebro.adddata(data)  # 将行情数据对象注入引擎
cerebro.addstrategy(SmaCross)  # 将策略注入引擎

cerebro.broker.setcash(10000.0)  # 设置初始资金

# 加入pyfolio分析者
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

results=cerebro.run()  # 运行
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
pf.create_full_tear_sheet(returns)

 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 10:10:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 10:30:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 11:40:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 12:20:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 13:35:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 14:20:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 14:35:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 16:30:00 订单 Canceled/Margin/Rejected
 Mon 2006-01-02 17:30:00 创建买单2006-01-02 17:30:00
 Mon 2006-01-02 17:10:00 订单 Canceled/Margin/Rejected


Start date,2006-01-02
End date,2006-01-02
Total months,0
,Backtest
Annual return,0.0%
Cumulative returns,0.0%
Annual volatility,nan%
Sharpe ratio,NaN
Calmar ratio,NaN
Stability,NaN
Max drawdown,0.0%


IndexError: index -1 is out of bounds for axis 0 with size 0

In [15]:
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

cerebro = bt.Cerebro()

dataname = '../data/2005-2006-day-001.txt'
data = btfeeds.BacktraderCSVData(dataname=dataname)
cerebro.adddata(data)
cerebro.addstrategy(btstrats.SMA_CrossOver)

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
import pyfolio as pf
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    live_start_date='2005-05-01',
    )

cerebro.plot()

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10496.68


C:\ProgramData\Anaconda3\envs\py39\lib\site-packages\empyrical\utils.py:440: UserWarning: Yahoo Finance read failed: mktime argument out of range, falling back to Google
  warnings.warn(


AttributeError: module 'pandas_datareader.data' has no attribute 'get_data_google'

In [18]:
import datetime

import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

cerebro = bt.Cerebro()

dataname = '../data/2005-2006-day-001.txt'
data = btfeeds.BacktraderCSVData(dataname=dataname)
cerebro.adddata(data)
cerebro.addstrategy(btstrats.SMA_CrossOver)


#……策略和参数
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
back = cerebro.run()
strat = back[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
print(returns)
returns.index = returns.index.tz_convert(None)
import quantstats
quantstats.reports.html(returns, output='stats.html', title='BTC Sentiment')



index
2005-01-03 00:00:00+00:00    0.000000
2005-01-04 00:00:00+00:00    0.000000
2005-01-05 00:00:00+00:00    0.000000
2005-01-06 00:00:00+00:00    0.000000
2005-01-07 00:00:00+00:00    0.000000
                               ...   
2006-12-21 00:00:00+00:00   -0.000614
2006-12-22 00:00:00+00:00   -0.003680
2006-12-27 00:00:00+00:00    0.005872
2006-12-28 00:00:00+00:00   -0.000400
2006-12-29 00:00:00+00:00   -0.001020
Name: return, Length: 512, dtype: float64
